#Dataset 

In [1]:
import pandas as pd
from datasets import Dataset 
import numpy as np


For my finetuned dataset, what I'm doing is download the top ten most popular jp novels from the site novelupdates.com. Then using the japanese title found on this page and downloading the japanse webnovel from syosetsu. While I could try and automate this part, I don't really have the front end experience to do so. That is also not the focus of this project.

So I realized that encoding the JP and the EN parts separately would mean they wouldn't be able to talk to each other. I created the file below to turn into my now reduced dataset

In [2]:
dataframe = pd.read_csv("C:/Users/Dylan/Downloads/syosetu/Sentence Pairs.txt", sep="|", header=None, encoding="utf-8")
dataframe.columns = ["translation", "EN_for_now"]
dataframe

,translation,EN_for_now
0,そして兄ブダリオンを超えたブギータスは、遂に帝位を奪い取った。そしてそのまま瞬く間に他の国を...,And having surpassed his older brother Budari...
1,地球での伯父も流石に白米を食べる事を贅沢だとは言わなかったので、当時は普通に白米を食していた。,His uncle on Earth hadn’t gone as far as to s...
2,しかし、オリジンで二度目の人生を過ごした軍事国家の研究所は、地球の欧州圏に該当する文化圏にあ...,"However, in the military nation in Origin tha..."
3,荷物の運搬の為に、嫌悪感を抑えて空間属性魔術を使うグーバモンのアンデッドを作るべきかと思った...,Vandalieu had been wondering whether he shoul...
4,しかしテイマーの女は諦めない。鞭を振るい、【限界超越】、【魔鞭限界突破】を発動。ただでさえ残...,She brandished her whip and activated ‘Transc...
5,『スキルとは、生物が持つ魂の力を簡単に引き出すために、魂の一部を改変したもの』,『A skill is the transformation of part of the...
6,「さきほどあなたがおっしゃったのは、ＭＡエネルギーだけに焦点を合わせたことです。この世界の住...,「Referring to what you said a short while ago...
7,繰り返す、分体は本体の思考をサポートし、口が滑らかに動くように尽力せよ！,"I repeat, the clones are to support the main ..."
8,《確認しました。対熱耐性獲得・・・成功しました》,<<Confirmed. Establishing heat resistance. Su...
9,時間をくれ、落ち着くから。こういう時は素数を数えたらいいんだっけ？,"Just an hour please, let me catch my breath. ..."


I ended up having to do some manipulation dataframe manipulation to get the tokenizer to recognize the dataset I created

In [3]:
dataframe["translation"] = "\"jp\": " + '"' + dataframe["translation"] + '"'
dataframe["EN_for_now"] = "\"en\": " + '"'  + dataframe["EN_for_now"] + '"'
dataframe["translation"] = '{' + dataframe["translation"] + ", " + dataframe["EN_for_now"] + '}'


In [4]:
dataframe.drop(["EN_for_now"], axis=1, inplace=True)

In [5]:
dataframe.head(11)

,translation
0,"{""jp"": ""そして兄ブダリオンを超えたブギータスは、遂に帝位を奪い取った。そしてそのまま..."
1,"{""jp"": ""地球での伯父も流石に白米を食べる事を贅沢だとは言わなかったので、当時は普通に..."
2,"{""jp"": ""しかし、オリジンで二度目の人生を過ごした軍事国家の研究所は、地球の欧州圏に該..."
3,"{""jp"": ""荷物の運搬の為に、嫌悪感を抑えて空間属性魔術を使うグーバモンのアンデッドを作..."
4,"{""jp"": ""しかしテイマーの女は諦めない。鞭を振るい、【限界超越】、【魔鞭限界突破】を発..."
5,"{""jp"": ""『スキルとは、生物が持つ魂の力を簡単に引き出すために、魂の一部を改変したもの..."
6,"{""jp"": ""「さきほどあなたがおっしゃったのは、ＭＡエネルギーだけに焦点を合わせたことで..."
7,"{""jp"": ""繰り返す、分体は本体の思考をサポートし、口が滑らかに動くように尽力せよ！ ""..."
8,"{""jp"": ""《確認しました。対熱耐性獲得・・・成功しました》 "", ""en"": "" <<..."
9,"{""jp"": ""時間をくれ、落ち着くから。こういう時は素数を数えたらいいんだっけ？ "", ""..."


In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ja-en", return_tensors="pt")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ja-en")

c:\Users\Dylan\mambaforge-pypy3\envs\fastai\Lib\site-packages\transformers\models\marian\tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [7]:
from datasets import load_dataset

testDataset = load_dataset("open_subtitles", "bn-is")

In [8]:
openDs = pd.DataFrame(testDataset["train"])

In [9]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
openDs.drop("meta", axis=1, inplace=True)
openDs

,id,translation
0,0,"{'bn': 'হবিটোস কাছে কোথাও আছে.', 'is': 'Eitrið..."
1,1,"{'bn': 'বিষ এখনো তাজা, তিন দিন ধরে.', 'is': 'Þ..."
2,2,"{'bn': 'তারা আমাদের পিছু নিয়েছে.', 'is': 'Ef þ..."
3,3,"{'bn': 'এটাই সেটা.', 'is': 'Hér komst Forresta..."
4,4,"{'bn': 'এটাকেই ফরেস্টাল খুঁজেছিল.', 'is': '- E..."
...,...,...
38267,38267,"{'bn': 'অস্ত্র ফেলে দাও!', 'is': 'Slepptu vopn..."
38268,38268,"{'bn': 'ফেলো! এক্ষণ!', 'is': 'Slepptu því núna!'}"
38269,38269,"{'bn': 'মাটিতে শুয়ে পড়ো!', 'is': 'Á jörðina nú..."
38270,38270,"{'bn': 'সিজ অল মোটর ফাংশনস!', 'is': 'Frystið a..."


In [10]:

dataset = Dataset.from_pandas(dataframe.applymap(str))

dataset

C:\Users\Dylan\AppData\Local\Temp\ipykernel_62240\2461113079.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dataset = Dataset.from_pandas(dataframe.applymap(str))


Dataset({
    features: ['translation'],
    num_rows: 17
})

In [11]:
#dataset["Translation"][0]
testDataset["train"]["translation"][200]

{'bn': 'আমরা... ...পিপাসার্ত নই.', 'is': '- Hvað er þér á höndum?'}

In [12]:
tokenizedTestDatasetTrain = testDataset["train"]


In [47]:
def tokenize_function(examples):
    return tokenizer(examples["translation"], padding="max_length", truncation=True)


In [48]:

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets
#tokenizedTestDatasetTrain = tokenizedTestDatasetTrain.map(tokenize_function, batched=True)

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

Dataset({
    features: ['translation', 'input_ids', 'attention_mask'],
    num_rows: 17
})

In [45]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [51]:
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.3)
tokenized_datasets["test"]["translation"]

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask'],
        num_rows: 11
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask'],
        num_rows: 6
    })
})

In [17]:
import evaluate

metric = evaluate.load("sacrebleu")

In [18]:
import torch
torch.cuda.is_available()

True

In [40]:

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

In [33]:
#trainer.train()

In [ ]:
trainer.train_dataset[1]

I couldn't get the trainer to work so I'm trying it in pytorch. All of the code below is copied from the huggingface translation example

In [35]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=8)
eval_dataloader = DataLoader(tokenized_datasets["test"], batch_size=8)

In [34]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [41]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"test_jp-en",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

In [38]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(60716, 512, padding_idx=60715)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(60716, 512, padding_idx=60715)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,),

In [52]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], collate_fn=data_collator, batch_size=8
)

In [53]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [54]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [55]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [57]:
output_dir = "jp_en_test"

In [58]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        #repo.push_to_hub(
         #   commit_message=f"Training in progress epoch {epoch}", blocking=False
        #)

  0%|          | 0/6 [00:00<?, ?it/s]

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`translation` in this case) have excessive nesting (inputs type `list` where type `int` is expected).